# Fine-tuning llama2 (Youri-7b)

* https://www.kaggle.com/code/philculliton/fine-tuning-with-llama-2-qlora
* https://www.kaggle.com/code/hhoang41/llama-2-fine-tuning-using-qlora
* https://colab.research.google.com/drive/1tG9eqttfnqHoQqmsiacywUG9ilUhoiCk?usp=sharing#scrollTo=5o3OgCMdRGgp

In [1]:
# ! pip install accelerate peft bitsandbytes transformers trl

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from bs4 import BeautifulSoup
import re

/home/peryaudo/mambaforge/envs/peryagpt/lib/python3.11/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [3]:
def cleanup_comment(comment):
    text = BeautifulSoup(comment).get_text(separator = '\n', strip = True)
    text = re.sub(r'http\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'>>[0-9]+', '', text, flags=re.MULTILINE)
    return text.strip()

In [4]:
def process_thread(examples):
    prompts = []
    for thread in examples['text']:
        comments = [cleanup_comment(line.split('<>')[3]) for line in thread.split('\n') if line]
        comments = comments[:999]
        prompts += ["ニュース:" + comments[0] + "\n5chの反応:" + comment for comment in comments[1:]]
    return {"text": prompts}

In [5]:
dataset = load_dataset("text", data_dir="scraped", sample_by='document')['train']

Resolving data files:   0%|          | 0/1194 [00:00<?, ?it/s]

In [6]:
dataset = dataset.map(process_thread, batched=True)
dataset = dataset.shuffle()
dataset = dataset.select(range(int(len(dataset) * 0.1)))

In [7]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    "rinna/youri-7b",
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained("rinna/youri-7b", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [10]:
peft_args = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [11]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [12]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_args,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/home/peryaudo/mambaforge/envs/peryagpt/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/61426 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,1.072600
50,1.010800
75,0.990100
100,0.946000


In [ ]:
# TODO: >>1 tends to be too long for the context length. Is it possible to truncate it beforehand?